# 라이브러리 정의

In [1]:
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

C:\Users\kopo\anaconda3\lib\site-packages\sklearn\externals\six.py:28: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  warnings.warn("The module is deprecated in version 0.21 and will be removed "


In [2]:
# 데이터 처리 라이브러리
import pandas as pd
import numpy as np

# 분석 알고리즘 DecisionTree 구현 라이브러리
# 설명력은 좀 낮지만 정확도 높음
from sklearn.tree import DecisionTreeRegressor

# 과거데이터를 8:2, 7:3이나 이런식으로 자동으로 나누어주는 라이브러리
from sklearn.model_selection import train_test_split 

# 라벨 인코더 -> 문자를 숫자로 맵핑시켜준다. 두개는 0,1 세개는 0,1,2 네개는 0,1,2,3
from sklearn.preprocessing import LabelEncoder

In [96]:
# slearn.ensemble안에 RandomForestRegressor라이브러리 
from sklearn.ensemble import RandomForestRegressor
# 분석 알고리즘 DecisionTree 구현 라이브러리
# 설명력은 좀 낮지만 정확도 높음
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [32]:
 # 분석 Regression!!   대표 평가지표  (MAE, RMSE)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error  

In [4]:
# from sklearn.ensemble import RandomForestRegressor
# 설명력이 뛰어남

# from sklearn.linear_model import LinearRegression
# 연속적 예측

In [33]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


# 데이터 불러오기

In [34]:
featuresData = \
    pd.read_csv("../dataset/feature_regression_example.csv")

# 1.데이터 전처리

### 1-1. 타입통합/ 특성 숫자컬럼 추가

### 1-1-1. 데이터 타입 통합

In [7]:
# featuresData.info()

In [35]:
# 주의할 사항은 모든 컬럼에 대해서 고정시키는걸 고려하는게 나을 수도 있다
featuresData.QTY = featuresData.QTY.astype(float)  # qty타입을 플롯으로 바꾼다

### 1-1-2. 특성 값 숫자컬럼 변경

In [36]:
#  이유 : 머신러닝 특성은 숫자형 지원
# 해결 : Y->1, N->0 과 같이 변환

In [37]:
featuresData.columns

Index(['REGIONID', 'PRODUCTGROUP', 'PRODUCT', 'ITEM', 'YEARWEEK', 'YEAR',
       'WEEK', 'QTY', 'HOLIDAY', 'HCLUS', 'PROMOTION', 'PRO_PERCENT'],
      dtype='object')

In [38]:
featuresData.PROMOTION.drop_duplicates()

0    Y
9    N
Name: PROMOTION, dtype: object

In [39]:
featuresData["HOLIDAY_NEW"] = np.where(featuresData.HOLIDAY=="Y",1,0)

In [40]:
featuresData["PROMOTION_NEW"] = np.where(featuresData.PROMOTION=="Y",1,0)


In [41]:
# 이 모양으로도 가능
# pd.DataFrame( featuresData.HOLIDAY.drop_duplicates() ) .reset_index().rename(columns={"index" : "HOLIDAY_ENCODE"})

# 1-2. 특성 선정/ 데이터 분리(특성 데이터 추출)

### 1-2-1. 특성선정

In [42]:
corrDf = featuresData.corr() # corr() -> 상관계수를 뽑을 수 있다 yearweek와 qty의 상관성을 알 수 있다.

In [43]:
standardLimit = 0.5

In [44]:
# 반응하는 변수들 
corrDf.loc[ (abs(corrDf.QTY) > standardLimit) & (corrDf.QTY !=1)]

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT,HOLIDAY_NEW,PROMOTION_NEW
HCLUS,-0.071586,0.028593,-0.339943,-0.537230,1.000000,-0.545619,-0.974601,-0.374072
PRO_PERCENT,0.404889,0.321193,0.329705,0.700195,-0.545619,1.000000,0.487062,0.898554
HOLIDAY_NEW,0.049867,-0.031106,0.273710,0.505932,-0.974601,0.487062,1.000000,0.365148
PROMOTION_NEW,0.205916,0.195931,0.060206,0.612451,-0.374072,0.898554,0.365148,1.000000


In [45]:
# standard limit에 따라 features가 바뀔 수 있음
features = \
    list( corrDf.loc[ (abs(corrDf.QTY) > standardLimit) & 
                     (corrDf.QTY != 1) ].index )

In [46]:
features

['HCLUS', 'PRO_PERCENT', 'HOLIDAY_NEW', 'PROMOTION_NEW']

In [47]:
label = ["QTY"]

### 1-2-2. 데이터 분리

In [48]:
# 문제지는 feattures 정답지는 label
# 과거실적 데이터를 7:3, 8:2정도로 분리를 시킨다 (70퍼센트 정도를 컴퓨터에게 훈련 시키고 나머지 30퍼셑느로 답을 비교함)

In [49]:
standardIndex = 0.8

In [50]:
featuresData.shape

(105, 14)

In [51]:
sortKey = ["REGIONID","ITEM", "YEARWEEK"]

In [52]:
sortedData = featuresData.sort_values(sortKey, ignore_index=True)

In [53]:
selectedIndex = int( (sortedData.shape )[0] * 0.8 ) 
# 정렬하고 80%에 있는 인덱스의 번호로 분리시킨다~!

In [54]:
yearweekStd = sortedData.loc[selectedIndex].YEARWEEK
yearweekStd

201632

In [55]:
# 훈련데이터와 테스트데이터를 (문제지와 정답지로 구분해서 정의한다.)(문제, 정답, 문제, 정답)
trainingDataFeatures = \
    sortedData.loc[sortedData.YEARWEEK <= yearweekStd, features] # 과거 문제지 80%
trainingDataLabel = \
    sortedData.loc[sortedData.YEARWEEK <= yearweekStd, label] # 과거 정답지 80%      과거 80퍼센트의 문제지와 정답지로 기계에 훈련(fit)시킨다 훈련->x 훈련결과y 
testDataFeatures = \
    sortedData.loc[sortedData.YEARWEEK > yearweekStd, features] #나머지20% 문제지 이걸 기계에 훈련시기면 나오는 답을 최신정답지와 비교하는것
testDataLabel = \
    sortedData.loc[sortedData.YEARWEEK > yearweekStd, label] # 최신 정답지  %20
                                                             # 가장 최신 정답지(20,30%)는 항상 비교군이다 (이미 최신 정답지는 있음, 과거의 피쳐를 돌려서 얼마나 정확하게 예측하는지 비교하기 위해서 필요한 것)

In [56]:
trainingDataFeatures

,HCLUS,PRO_PERCENT,HOLIDAY_NEW,PROMOTION_NEW
0,1,0.209442,1,1
1,4,0.209442,0,1
2,4,0.208155,0,1
3,2,0.208155,1,1
4,4,0.208155,0,1
...,...,...,...,...
80,1,0.308584,1,1
81,1,0.308584,1,1
82,4,0.280258,0,1
83,4,0.280258,0,1


# 2. 모델적용

### 2-1. 모델적용

### 2-1-1. 학습

In [100]:
# 모델 선언(Decision, Random)
model_dt = DecisionTreeRegressor(random_state=10) # 난수를 고정, 난수를 발생시키면서 스무고개가 시작됨 이걸 고정안해주면 트리가 계속 바뀜
model_rf = RandomForestRegressor(random_state=10) # (안쓰면 기본 디폴트 100개)
model_lr = LinearRegression()

In [102]:
# 훈련     훈련을 시키고 예측을 한다
# 훈련이 fit!! 훈련함과 동시에훈련결과가  바로 할당이 된다
model_dt.fit(X=trainingDataFeatures, y=trainingDataLabel)
model_rf.fit(X=trainingDataFeatures, y=trainingDataLabel)
model_lr.fit(X=trainingDataFeatures, y=trainingDataLabel)

<ipython-input-102-761ebaa4c7c1>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf.fit(X=trainingDataFeatures, y=trainingDataLabel)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# 3. 예측

In [104]:
predictValueDt =  model_dt.predict(testDataFeatures)

In [105]:
predictValueRf = model_rf.predict(testDataFeatures)

In [106]:
predicValueLr = model_lr.predict(testDataFeatures)

In [107]:
testDataAll = featuresData.loc[ testDataFeatures.index ]

In [108]:
testDataAll = pd.DataFrame(predictValueDt, columns=["PREDICT_DT"])
testDataAll = pd.DataFrame(predictValueRf, columns=["PREDICT_Rf"])

In [93]:
testDataAll

,PREDICT_Rf
0,353.535465
1,1409.048303
2,1409.048303
3,1409.048303
4,1439.099008
5,1439.099008
6,1731.856825
7,1731.856825
8,2576.315483
9,1731.856825


In [39]:
# predictValue = model.predict(testDataFeatures)

In [186]:
# predictValue

array([ 350.71428571, 1434.6       , 1434.6       , 1434.6       ,
       1434.6       , 1434.6       , 1708.75      , 1708.75      ,
       2620.42857143, 1708.75      ,  350.71428571, 2193.8       ,
       2193.8       , 2193.8       , 2193.8       , 2193.8       ,
       1434.6       , 1434.6       , 1434.6       , 1434.6       ])

# 4. 데이터 정리

In [ ]:
 # 3.예측 과 4. 데이터정리  를 데이터 전처리로 합친다

In [81]:
validateDf = \
    pd.concat( [testDataLabel.reset_index(drop=True), predictDtDf, predictRfDf ], axis = 1)

# 5. 정확도 검증

In [82]:
# mae = mean_absolute_error( y_true=validateDf.QTY,  
#                    y_pred=validateDf.PREDICT )

AttributeError: 'DataFrame' object has no attribute 'PREDICT'

In [190]:
# rmse = np.sqrt( mean_squared_error(y_true = validateDf.QTY, 
#                            y_pred=validateDf.PREDICT) )

In [98]:
### 장점
### DecisionTree -> 과거의 겅험치를 그대로 반영한다~!
###                 변동성이 큰 데이터에서 강하다~!

### 단점
### 오버피팅 문제가 생김!! -> 너무 과거에 얽매여서 미래에대한 어떠한 포인트도 줄 수 없다. (과거에 너무 심취ㅋ ex)한국사시험 p84쪽 두번째 줄에 있던 내용에 집중하는 고런 너낌)
### randomforest -> 
### 장점 : DecisionTree의 오버피팅을 해결하고
### 단점 : 설명력이 좀 어렵다~

In [84]:
features

['HCLUS', 'PRO_PERCENT', 'HOLIDAY_NEW', 'PROMOTION_NEW']

In [85]:
# 대휴일 -> 1   소휴일 -> 4
userInputHCLUS = 1
# 제품할인 %
userInputProPercent = 0.5
# 홀리데이 유뮤 Y = 1 N = 0
userInputHoilidayYn = 1  # "Y"
# 프로모션 유무 Y = 1 N = 0
userInputPromotionYn = 1 # "Y"

In [86]:
futereData=pd.DataFrame( [userInputHCLUS,userInputProPercent,userInputHoilidayYn, userInputPromotionYn])

In [87]:
# pickle파일로 저장 가능하다  futerData를 모델디티로 프레딕트를 이용해서 
model_dt.predict(futureData)

NameError: name 'futureData' is not defined

In [ ]:
model_rf.

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

export_graphviz(decision_tree=model_dt,
                   out_file="tree.dot",
               impurity=True)
with open("tree.dot")as f:
    dot_graph = f.read()
display(graphviz.Source(dot_graph))

In [ ]:
treeStep1 = traningDataFeatures.loc[trainingDataFeatures.PRO_PERCEN > 0.259]

treeStep2 = treeStep1.loc[treeStep1.PRO_PERCENT > 0.294]

treeStpe3 = treeStep2.loc[treeStep2.HCLUS > 0.5]

treeStpe3

treeStep3.index

trainingDataLabel.loc[treeStep3.index].QTY.mean()

In [ ]:
## 참고
## 머신러닝/딥러닝!!
## lable 인코더 -> 문자 -> 숫자로 맵핑시켜준다 